1. Анализ данных из таблиц transaction и customer 

Таблица customer содержит информацию о клиентах:
- для каждого customer_id есть блок личной информации, однозначно характеризующей клиента (фамилия, имя, пол, дата рождения); 
- адрес (страна, штат, индекс, address) можно отнести к характеристике клиента (т.к. обычно указывается один адрес) или можно вынести в отдельную таблицу адресов - это поможет избежать дублей, если по одному адресу зарегистрировано несколько клиентов; также есть проблема с данными в address - склеены номера домов и улицы (а для нормализации их нужно разъединить). Разъединить address пока не получится. Страна везде одна (выглядит, как избыточное значение, особенно, если компания работает исключительно в Австралии), городов вообще нет. Не вижу смысла выделять адреса в отдельную таблицу
- отдельно напрашивается блок о работе (сфера деятельности, профессия)
- также есть некая дополнительная информация о клиенте (deceased_indicator - признак смерти; owns_car - наличие машины; wealth_segment - категория клиента для сегментирования). Т.к. данных немного, можно добавить их в блок с основной информацией или вынести в отдельную; 
- property_valuation - видимо, оценка недвижимости клиента, тогда она связана с адресом

Таблица transaction содержит несколько смысловых блоков:
- именно к транзакции относится transaction_id, можно отнести customer_id в качестве внешнего ключа, дату транзакции и нужна сумма транзакции - нет ясности какой из атрибутов является именно суммой транзакции, но у нас есть list_price и standard_cost (нужно подумать, к чему они относятся)
- хочется выделить сущность продукт, куда можно было бы отнести атрибуты brand, product_line (какая-то характеристика линейки), product_class (класс продукта), product_size (размеры), НО для одних и тех же продуктов эти значения разные (видно по product_id) - пока сомневаюсь, стоит ли создавать много мини-таблиц с одним атрибутом внутри, чтобы положить в products кучу идентификаторов
- можно выделить сущность заказ, куда войдет информация о характеристиках покупаемого товара, а также признак online_order, order_status

2. Нормализация БД

Данные в таблицах выглядят запутанно; перемешана и слеплена логика; одни и те же по смыслу значения сохранены в разном виде (например, gender содержит вместо структурированных значений вариации)
Данные содержат пропуски; не везде используется корректный формат
Таблицы не удовлятворяют требованиям 1НФ, т.к. данные не атомарны, есть повторяющиеся группы
Для 2НФ необходимо, чтобы неключевые атрибуты зависели от первичного ключа
Для 3НФ требуется устранение транзитивных зависимостей

1) разобъем 2 исходные таблицы на несколько более мелких, чтобы избежать дублирований в строках
2) для новых мелких таблиц определим первичные ключи
3) настроим связи между таблицами
- customers - customer_id (PK), first_name, last_name, birthdate, gender_id (FK), job_category_id (FK), job_id (FK), wealth_segment_id (FK)
- genders - gender_id (PK), gender_name
- countries - country_id (PK), country_name
- states - state_id (PK), state_name
- addresses - address_id (PK), post_code, address, country_id (FK), state_id (FK) 
- job_categoties - job_category_id (PK), job_industry_category
- job_titles - job_id (PK), job_title
- wealth_segments - wealth_segment_id (PK), wealth_segment
- deceased_indicators - deceased_indicator (PK), deceased_indicator
- owns_cars - owns_car_id (PK), owns_car
- transactions - transaction_id (PK), transaction_date, customer_id (FK), order_id (FK)
- products - product_id (PK), brand_id (FK)
- orders - order_id (PK), online_order, order_status, product_id (FK), price_id (FK)
- product_lines - product_line_id (PK), product_line_name
- brands - brand_id (PK), brand_name
- product_classes - class_id (PK), product_class
- product_sizes - size_id (PK), product_size
- prices - price_id (PK), list_price, standard_cost

В общем, я запуталась) и мне не нравятся эти таблицы, потому что они выглядят странно. Точно не хочу создавать множество "справочников" из id и одного значения. И не до конца уловила смысл некоторых атрибутов

3. Моя финальная таблица:
- customers - customer_id (PK), first_name, last_name, birthdate, address_id (FK), job_category_id (FK), job_id (FK), wealth_segment, gender_name, deceased_indicator, owns_car
- countries - country_id (PK), country_name
- states - state_id (PK), state_name
- addresses - address_id (PK), post_code, address, country_id (FK), state_id (FK) 
- job_categoties - job_category_id (PK), job_industry_category
- job_titles - job_id (PK), job_title
- transactions - transaction_id (PK), transaction_date, customer_id (FK), order_id (FK), list_price, standard_cost
- products - product_id (PK), brand_name
- orders - order_id (PK), online_order, order_status, product_id (FK), customer_id (FK), price_id (FK), product_line_id (FK), class_id (FK), size_id (FK)
- product_lines - product_line_id (PK), product_line_name
- product_classes - class_id (PK), product_class
- product_sizes - size_id (PK), product_size

Этот вариант точно не приведен к 3НФ, и до сих пор мне не нравится (смущает, что отсутствуют некоторые важные атрибуты; и не очень укладываются в красивую картинку БД)